In [ ]:
!rm Train.csv
fname="https://github.com/sipocz/malavi/raw/ccefa7562b0a115d207a02f2cf2754406870ef52/Train.csv"
!wget https://github.com/sipocz/malavi/raw/ccefa7562b0a115d207a02f2cf2754406870ef52/Train.csv

In [ ]:
!head Train.csv

In [ ]:
import pandas as pd
df=pd.read_csv("Train.csv",sep=",")

In [ ]:
len(df)

In [ ]:
labels=['SOCIAL ISSUES', 'EDUCATION', 'RELATIONSHIPS', 'ECONOMY', 'RELIGION', 'POLITICS', 'LAW/ORDER', 'SOCIAL', 'HEALTH', 'ARTS AND CRAFTS', 'FARMING', 'CULTURE', 'FLOODING', 'WITCHCRAFT', 'MUSIC', 'TRANSPORT', 'WILDLIFE/ENVIRONMENT', 'LOCALCHIEFS', 'SPORTS', 'OPINION/ESSAY']

In [ ]:
labels

In [ ]:
label_index=[]
for i in list(df.Label):
    label_index.append(labels.index(i))

In [ ]:
df["label_index"]=label_index

In [ ]:
df.head()

In [ ]:
s1=set()
for i in range(len(df)):
    s1=s1.union(set(df.Text[i].replace(",","").replace(".","").replace("?","").replace("(","").replace(")","").replace("!","").replace(";","").lower().split()))
words=list(s1)

In [ ]:
words.sort()

In [ ]:
words[10:100]

In [ ]:
def findA():
    for inx,i in enumerate(words):
        if i=="a":
            print(inx)
            return(inx)
words=words[findA():]

In [ ]:
wlen=99

In [ ]:
m6=[]
for w in words:
    if len(w)>wlen:
        m6.append(w[:wlen])
    else:
        m6.append(w)
m6=list(set(m6))



In [ ]:
m6.sort()

In [ ]:
m6[:200]

In [ ]:

lines=[]
for i in range(len(df)):
    s1=set(df.Text[i].replace(",","").replace(".","").replace("?","").replace("(","").replace(")","").replace("!","").replace(";","").lower().split())
    line=[]
    for w in s1:
        if len(w)>wlen:
            wmini=w[:wlen]
        else:
            wmini=w
        if wmini in m6:
            line.append(m6.index(wmini))
    lines.append(line)

In [ ]:
len(lines[0])

In [ ]:
ritka=[]

for line in lines:
    ritkaline=[0 for i in range(len(m6))]
    for pos in line:
        ritkaline[pos]=ritkaline[pos]+1
    ritka.append(ritkaline)

In [ ]:
!pip install tensorflow-addons

In [ ]:
from keras.layers import InputLayer, Dense, LSTM, Input, Dropout
from keras.models import Sequential, Model
from keras.optimizers import SGD,Adam,Adamax,Nadam,Ftrl,Adadelta,Adagrad,Nadam
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.backend import clear_session
from tensorflow.keras.losses import mean_absolute_percentage_error, huber,kld
import tensorflow_addons as tfa


In [ ]:
len(ritka)

In [ ]:
divi=int(len(ritka)*0.95)

In [ ]:
X=ritka[:divi]
X_test=ritka[divi:]
yall=list(df.label_index)
y=yall[:divi]
y_test=yall[divi:]

In [ ]:
len(X_test)

In [83]:
def furesz( epoch,lr, range=500,lrmin=0.000001,lrmax=0.0001):
    lrcalc=lrmax*((epoch%range)/range)+lrmin
    print(f"lrcalc:{lrcalc:.6}, epoch: {epoch}")
    return(lr)
    

In [84]:
def scheduler(epoch, lr):
    lr_o=furesz(epoch,lr)
    return(lr_o)

In [85]:
callback_LR = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
clear_session()

kernel_reg_1=tf.keras.regularizers.L2(0.1)

input_size=len(X[0])
output_size=20
drop_frac0=0.0 
drop_frac1=0.90




input1=Input(shape=(input_size,))

l0_out=Dropout(drop_frac0)(input1)
l1_out=Dense(50,activation="tanh",kernel_regularizer="L1L2")(l0_out) # 202102262002 l1_out=Dense(370,activation="sigmoid")(input1)
l2_out=Dropout(drop_frac1)(l1_out)





pred=Dense(output_size,activation="softmax")(l2_out)

model = Model(inputs=input1, outputs=pred)
optimizer=Adamax(learning_rate=0.00005,) #202102261859 ~0,0038: Adamax(learning_rate=0.01,) #SGD(lr=0.005, decay=1e-4, momentum=0.9)
optimizer=Nadam(learning_rate=0.00005,)

optimizer=tfa.optimizers.LazyAdam(0.001)
opt = tf.keras.optimizers.Adam(learning_rate=0.000003)
optimizer = tfa.optimizers.Lookahead(opt)

model.compile(loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=["accuracy"])

In [ ]:
fname="./a_model_rew_maxlen_3"

In [ ]:
callbacks = [callback_LR,
       
        ModelCheckpoint(filepath=fname+"_{loss:.5f}_{val_accuracy:.5f}_.hdf5", monitor='val_accuracy',
                        verbose=2, save_best_only=True, mode='max')]

In [ ]:
history = model.fit( X,y,validation_data=(X_test,y_test), 
                        epochs=1500, 
                        batch_size=3,
                        
                        verbose=1,
                        callbacks=callbacks
                              
                              )

Epoch 1/1500
lrcalc:1e-06, epoch: 0
455/455 [==============================] - 20s 43ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 2.1354 - val_accuracy: 0.6111

Epoch 00001: val_accuracy did not improve from 0.63889
Epoch 2/1500
lrcalc:1.2e-06, epoch: 1
455/455 [==============================] - 3s 7ms/step - loss: 7.0962e-04 - accuracy: 1.0000 - val_loss: 2.1366 - val_accuracy: 0.6111

Epoch 00002: val_accuracy did not improve from 0.63889
Epoch 3/1500
lrcalc:1.4e-06, epoch: 2
455/455 [==============================] - 3s 7ms/step - loss: 7.2686e-04 - accuracy: 1.0000 - val_loss: 2.1365 - val_accuracy: 0.6111

Epoch 00003: val_accuracy did not improve from 0.63889
Epoch 4/1500
lrcalc:1.6e-06, epoch: 3
455/455 [==============================] - 3s 7ms/step - loss: 9.8528e-04 - accuracy: 0.9993 - val_loss: 2.1365 - val_accuracy: 0.6111

Epoch 00004: val_accuracy did not improve from 0.63889
Epoch 5/1500
lrcalc:1.8e-06, epoch: 4
455/455 [==============================] - 3s 7ms/

In [ ]:
model.load_weights("./a_model_rew_maxlen_3_0.18637_0.66667_.hdf5")

In [ ]:
!rm Test.csv
fname="https://github.com/sipocz/malavi/raw/main/Test.csv"
!wget https://github.com/sipocz/malavi/raw/main/Test.csv

In [ ]:
!head Test.csv

In [ ]:
df_test=pd.read_csv("Test.csv",sep=",")

In [ ]:
s1=set()
for i in range(len(df_test)):
    s1=s1.union(set(df_test.Text[i].replace(",","").replace(".","").replace("?","").replace("(","").replace(")","").replace("!","").replace(";","").lower().split()))
words_test=list(s1)

In [ ]:
lines=[]
for i in range(len(df_test)):
    s1=set(df_test.Text[i].replace(",","").replace(".","").replace("?","").replace("(","").replace(")","").replace("!","").replace(";","").lower().split())
    line=[]
    for w in s1:
        if len(w)>wlen:
            wmini=w[:wlen]
        else:
            wmini=w
        if wmini in m6:
            line.append(m6.index(wmini))
    lines.append(line)

In [ ]:
ritka_test=[]

for line in lines:
    ritkaline=[0 for i in range(len(m6))]
    for pos in line:
        ritkaline[pos]=1
    ritka_test.append(ritkaline)

In [ ]:
y_pred=model.predict(ritka_test)

In [ ]:
y_pred=list(y_pred)

In [ ]:
y_pred[:3]

In [ ]:
y_becs=[list(y_pred[i]).index(y_pred[i].max()) for i in range(len(y_pred))]  

In [ ]:
y_becs[:3]

In [ ]:
labels_res=[labels[y_becs[i]] for i in range (len(y_becs)) ]

In [ ]:
labels_res[:3]

In [ ]:
out=pd.DataFrame(data={"ID":df_test.ID,"Label":labels_res})

In [ ]:
out.head()

In [ ]:
out.to_csv("submission_9",index=False)

In [ ]:
!head submission_5